<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

### How can I create technical chart analysis of real time price feeds?

In this notebook, we will be creating a simple technical analysis chart of real time price feeds. It will include rates of change for prices overlayed by housing demand metrics via home purchases.

#### What will you create in this notebook?

<p align="center">
  <img src="../../images/technical_pf_usa_vs_demand_chart.png" alt="Alt text">
</p>

#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/price_feed_trader_resources/trader_charts.ipynb)

### 1. Import the Parcl Labs Python Library

In [2]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs nbformat

In [3]:
import os
import numpy as np
import pandas as pd
from parcllabs import ParclLabsClient
from parcllabs.beta.ts_stats import TimeSeriesAnalysis
from parcllabs.beta.charting.name_formats import format_names, create_ticker
from parcllabs.beta.charting.technical_charts import build_technical_chart

client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=12 # set default limit
)

### 2. Search for Price Feed Markets on Parcl Exchange

In [4]:
sales_pricefeed_markets = client.search.markets.retrieve(
    sort_by='PARCL_EXCHANGE_MARKET',
    sort_order='DESC',
    limit=15
)

sales_pricefeed_markets['clean_name'] = sales_pricefeed_markets['name'].apply(format_names)
sales_pricefeed_markets['ticker'] = sales_pricefeed_markets['clean_name'].apply(create_ticker)

### 3. Load the data for sales price feeds and sales demand

In [5]:
# lets retrieve data back to 2011 for these price feeds
START_DATE = '2019-01-01'
sales_price_feeds = client.price_feed.price_feed.retrieve(
    parcl_ids=sales_pricefeed_markets['parcl_id'].tolist(),
    start_date=START_DATE,
    limit=1000,  # expand the limit to 1000, these are daily series
    auto_paginate=True, # auto paginate to get all the data - WARNING: ~6k credits can be used in one parcl price feed. Change the START_DATE to a more recent date to reduce the number of credits used
)

|████████████████████████████████████████| 15/15 [100%] in 4.6s (3.27/s) 


In [6]:
# get supply and demand
demand = client.market_metrics.housing_event_counts.retrieve(
    parcl_ids=sales_pricefeed_markets['parcl_id'].tolist(),
    start_date=START_DATE,
    limit=1000,  # expand the limit to 1000, these are daily series
)

|████████████████████████████████████████| 15/15 [100%] in 1.2s (12.51/s) 


### Prepare the data for analysis/charting

In [7]:
# lets analyze las vegas city with parcl_id: 5377230
# we can confirm via:
las_vegas_pid = 5377230
sales_pricefeed_markets[sales_pricefeed_markets['parcl_id'] == las_vegas_pid]

,parcl_id,country,geoid,state_fips_code,name,state_abbreviation,region,location_type,total_population,median_income,parcl_exchange_market,pricefeed_market,case_shiller_10_market,case_shiller_20_market,clean_name,ticker
3,5377230,USA,3240000,32,Las Vegas City,NV,MOUNTAIN,CITY,644835,66356,1,1,0,0,Las Vegas City,LAS


In [8]:
lv_pricefeed = sales_price_feeds[sales_price_feeds['parcl_id'] == las_vegas_pid].copy()
lv_demand = demand[demand['parcl_id'] == las_vegas_pid].copy()
# sort ascending order
lv_pricefeed = lv_pricefeed.sort_values('date')
lv_demand = lv_demand.sort_values('date')
lv_metadata = sales_pricefeed_markets[sales_pricefeed_markets['parcl_id'] == las_vegas_pid]
ticker = lv_metadata['ticker'].values[0]
name = lv_metadata['clean_name'].values[0]

pf_ts_analysis = TimeSeriesAnalysis(lv_pricefeed, 'date', 'price_feed', freq='D')
pf_rate_of_change_stats = pf_ts_analysis.calculate_changes()

demand_ts_analysis = TimeSeriesAnalysis(lv_demand, 'date', 'sales', freq='M')
demand_rate_of_change_stats = demand_ts_analysis.calculate_changes()

In [9]:

# get last information for pricefeed
last_pricefeed_value = pf_rate_of_change_stats['last']['value']
last_pricefeed_date_str = pf_rate_of_change_stats['last']['date']
last_pricefeed_date = pd.to_datetime(last_pricefeed_date_str).strftime('%-d %B')
last_pf_date = pd.to_datetime(last_pricefeed_date_str).strftime('%-d-%B-%Y')

# get last information for demand
last_demand_value = demand_rate_of_change_stats['last']['value']
last_sales_month = pd.to_datetime(demand_rate_of_change_stats['last']['date']).strftime('%b')

In [10]:
# calculate daily return
lv_pricefeed['daily_return'] = lv_pricefeed['price_feed'].pct_change()
# drop na values
lv_pricefeed = lv_pricefeed.dropna().reset_index(drop=True)
lv_pricefeed['month'] = lv_pricefeed['date'].dt.month
lv_demand['month'] = lv_demand['date'].dt.month
# start and end of the month change
monthly_returns = lv_pricefeed.groupby('month')['price_feed'].agg(lambda x: x.iloc[-1] / x.iloc[0] - 1).reset_index(name='volColor')

In [11]:
# merge monthly returns
lv_pricefeed = lv_pricefeed.merge(monthly_returns, on='month', how='left')
lv_demand = lv_demand.merge(monthly_returns, on='month', how='left')

In [12]:
# Add volColor column directly to data_secondary
lv_demand['volColor'] = np.where(lv_demand['volColor'] >= 0, 'green', 'red')

In [13]:
# chart labels
msg = f"""
<b>52w High</b> {pf_rate_of_change_stats['52_week_high']['value']:.2f} 
<b>52w Low</b> {pf_rate_of_change_stats['52_week_low']['value']:.2f} 
<b>Last</b> {pf_rate_of_change_stats['last']['value']:.2f} 
<b>YoY Change</b> {pf_rate_of_change_stats['12_month']['value_diff']} ({pf_rate_of_change_stats['12_month']['percent_change']*100:.02f}%)
"""
msg = msg.replace('\n', '')


ticker_msg = f"<b>{ticker}</b> {name} Parcl Exchange"
volume_msg = f'Sales Volume (Monthly) {last_demand_value:,} ({last_sales_month})'
pricefeed_msg = f'{ticker} (Daily) ${last_pricefeed_value} ({last_pricefeed_date})'

In [14]:
build_technical_chart(
    line_chart_df=lv_pricefeed,
    bar_chart_df=lv_demand,
    value_name_main='price_feed',
    value_name_secondary='sales',
    header_left_primary_text=ticker_msg,
    header_left_secondary_text=last_pf_date,
    header_right_primary_text=msg,
    line_chart_left_primary_text=pricefeed_msg,
    bar_chart_left_primary_text=volume_msg,
    # save_path='lv_chart.png',
)